# [NCTS] FinalProject_Report : 預測指定商店特定商品之下個月銷售量

### 組名:銷售預測界SHE
組員:
* R07546016 張雯婷
* R07546014 陳柔君
* R07546001 邱諭宣

組內分工如下圖:

<img src="組內分工圖示.png" width="700" height="700" align=center>

### 專案進度排程:

<img src="期末排程.png" width="700" height="700" align=center>

### 目錄
1. 專案說明
2. 資料來源
3. 資料前處理
4. 模型成長歷史(微悲劇慎入)
5. 最終成功模型展示
6. kaggle競賽結果
7. 結論及未來實務應用

<img src="庫存示意圖.jpg" width="350" height="350" align=center>

### 1.專案說明
    *專案目的:店家在進行商品叫貨的時候，常會猶豫不同商品這個月要批多少，尤其在該商品不能久放的時候這問題會更加嚴重。
    *專案說明:藉由kaggle上的商店商品銷售紀錄資料，預測指定商店特定商品之下個月銷售量，以便店家在批貨的時候能夠有一個判斷的準則，進一步也可以透過此結果，預測該店家下個月的營業額，
    *資料集敘述如下:
    
* Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

### 2. 資料來源
[Kaggle_PredictFutureSales_Dataset](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data?fbclid=IwAR2AmvTp5RA3vihsRvqDrMYDCrhjlSwEfyWvOS76_jiUZ49X_-wEYDQaJyo)
<img src="kaggle.png" width="200" height="200" align=center>

### 3.資料前處理
#### 3-1.定義模型的輸入、輸出
    *資料集所包含的歷史資料涵蓋2013年一月至2015年十月，共34個月，由於X為欲預測月份之先前銷售資料，故可作為X的僅有33個月(無第35個月的銷售資料，故第34個月無法當作X)。
    *本專案33個月的訓練資料切割為31、2分別作為training data及validation data，以利於進行testing前的驗證評估。
    *以下為input、output資料格式說明。
    1.input data:
        每個row代表特定商店之指定商品一個月的每日銷售數量(在此將每個月皆預設為31天)。
        而商品及店家的排列組合共有424124個，每個組合又會丟進31個月，故input data row 為 (424124x31) ，input data column 為 31 。
    2.output data:
        資料中一個row代表該row對應在input data中該row的該店之指定商品在下個月的銷售量。
    3.模型應用:
        最後僅需要給予模型我們所要預測的商店之商品在該月的每日銷售量，就可以預測下個月該商品的銷售量。
#### 3-2.資料前處理實作程式碼
[DataPreprocess](https://nbviewer.jupyter.org/github/b3l4k5y67/My-Python-Fun-Lab/blob/master/PredictFutureSales_DataPreprocess.ipynb)

### 4. 模型成長歷史(微悲劇慎入)
    *由上步驟將資料前處理完成後，丟入模型進行訓練及預測分析。
#### 4-1.模型基礎架構說明
    本專案的模型架構皆分為兩個區塊，說明如下:
* 第一區塊:LSTM 
    將資料特定商店之該商品當月的每日銷售紀錄，丟入LSTM進行訓練，目標為藉由"時間序列"的規則訓練，獲取第一份結果。
* 第二區塊:NN
    小組內討論後認為，僅藉由時間序列的規則做為評估標準，可能無法更符合本專案希望預測該店家及指定商品之銷售的目的，故在此步驟加入NN，透過上一步驟LSTM之"時間序列分析結果"以及"店家ID"、"商品ID"，進行NN的分析，期望提升預測準確度。

#### 4-2.模型發想及建置    
    *以下統整嘗試過的模型、分析失敗原因，並提出解決方法。
    
    1.模型一、有使用active function的LSTM後續混合NN
        *結果:準確率4.9%
        *討論:跑完LSTM及NN後的準確率僅有4.9%，但過程中可以發現其實在僅使用LSTM，尚未使用NN架構時，模型竟達92%準確率，當時也近一步察看結果是否真的這麼優秀，發現準確率這麼高的原因是因為預測結果值都一樣(推測是因為訓練資料中區分店家及商品導致 該店家的該商品在該日的銷售量為0的狀況很多)，而這個相同的值剛好跟正確答案蠻近的，所以算下來準確很高。
        而後來也發現，由於預測結果應該包含正整數的銷售量，而非機率，但模型一使用softmax會導致出來的值介於0、1，這樣不符合所需。
        *解決方案:試著調大LSTM的數量，看能否更精確抓到時間序列的規則，並移除NN架構中的所有激活函數試看看
    
    2.模型二、沿用模型一，但調大LSTM個數，並將LSTM及NN架構皆移除激活函數
        *結果:準確率3.9%
        *討論:這次的預測結果已跳脫所有值都相同的窘境!但結果還是不佳!
        *解決方案:試著更換原本optimizer ADAM為SGD
        
    3.模型三、沿用模型二，但將optimizer 換為 SGD試試看
        *結果:準確率0%
        *討論:由於SGD為較陽春的優化器，進行訓練後，可與model2比較，還是使用原來的ADAM效果較佳。
        *解決方案:決定以模型二作為基礎，調整參數，並嘗試出最好的參數配置。
        
#### 4-3.模型建構實作程式碼
[ModelHistory](https://nbviewer.jupyter.org/github/b3l4k5y67/My-Python-Fun-Lab/blob/master/PredictFutureSales_ModelLose.ipynb)

### 5. 最終成功模型展示
#### 5-1.架構說明
    *最終成功模型是由模型三不斷調整參數進而完成的，詳細配置如5-2:
    *結果:準確率83%
#### 5-2.實作程式碼 (內涵結果分析圖)
[bestModel](https://nbviewer.jupyter.org/github/b3l4k5y67/My-Python-Fun-Lab/blob/master/PredictFutureSales_BestModel.ipynb)

### 6. kaggle競賽結果
    *將我們對於kaggle競賽上的testing data進行預測後繳交，並進行競賽評比，最終獲得1.34416的分數，由於每位參賽者個分數都相當接近，第一名的成績約0.7，而我們獲2580的名次。
    *競賽成果如下圖:
<img src="kaggle排名截圖.png" width="900" height="900" align=center>

### 7. 結論及未來實務應用
#### 7-1.結論
    *在本次專案研究中的模型實驗，我們驗證了許多在神經網絡的應用，如在優化器的選擇上，ADAM是現今最流行且穩定的選擇，而我們在調控優化器的過程中也發現了，使用ADAM確實能讓我們的結果較為精確。而模型的架構上，我們構過採取LSTM去獲取時間序列的特性，並以NN再將商家、商品的資訊納入分析的考慮因素，進行兩階段的分析，也讓結果更符合預測所需。
    *本堂課程讓我們對於人工智慧有了更深的了解及實務經驗。在專案中，我們能夠先對於row data先進行了解、討論該以什麼樣的資料結構去進行分析，並且構想模型架構的建置，過程中也在失敗中探討出可能的原因並進行修正，即使最終成果並非名列前茅，且架構及資料結構都仍須檢討，但學習過程中的實戰經驗卻是十分可貴的!
    *許多人都說神經網絡像個黑盒子，是個謎、無法解釋，但在本堂課的學習最後，我們認為這個黑盒子有時候是可以透過函數的理解、架構的分析，去進行可解釋性的調控，以獲得更好的結果。
    
#### 7-2.未來實務應用
    *由本專案之結果，我們提出以下兩項未來的實務應用面。
        *1.透過將欲預測商家的商品歷史銷售數據，可預測未來一個月的銷售量，對於保存期限不長的商品而言，無法一次叫過量的貨品作為存貨預防，便可藉此當作叫貨數量依據，避免屯貨過多造成浪費，或是存貨過少而不構的窘境。
        *2.投資人可以藉此預測結果的銷售量，進而取得預測總銷售額，了解該商店在未來的營利發展，當作投資者在投資商家時的考量指標之一。
        
<img src="錢.png" width="700" height="700" align=center>